In [35]:
import json
import csv
import requests
import os

# begin_time = datetime.datetime.now()

prop_lists_json_path = 'data/prop_lists/json/'
prop_lists_csv_path = 'data/prop_lists/csv/'
access_token = 'MjY0OTEw|6e27c6250a764bbfa6988a18ea93b4f3'
year = '2019'

city_id_dict = {
#     '79752' : 'dallas',
#     '79888' : 'fort-worth',
#     '80413' : 'plano',
#     '80045' : 'irving',
#     '79491' : 'arlington',
#     '79901' : 'frisco',
#     '79949' : 'grand-prairie',
#     '80237' : 'mckinney',
    '79771' : 'denton',
#     '80482' : 'richardson'
}

markets = {
    'dallas' : 'Dallas',
    'fort-worth' : 'Fort-Worth',
    'plano' : 'Plano',
    'irving' : 'Irving',
    'arlington' : 'Arlington',
    'frisco' : 'Frisco',
    'grand-prairie' : 'Grand Prairie',
    'mckinney' : 'McKinney',
    'denton' : 'Denton',    
    'richardson' : 'Richardson'
}

def setUp():      
    # Setup property listing folders
    if not os.path.exists(prop_lists_json_path):
        os.makedirs(prop_lists_json_path)
    if not os.path.exists(prop_lists_csv_path):
        os.makedirs(prop_lists_csv_path)

def fetch_prop_listing_json_files():    
    for city_id, city_name in city_id_dict.items():
        url='https://api.airdna.co/v1/market/property_list?access_token={}&city_id={}&start_month=1&start_year={}&number_of_months=12&currency=native&show_regions=true'.format(
            access_token, city_id, year
        )
        prop_list_json = requests.get(url)
        data = prop_list_json.content
        file_path = '{}{}_property_list_{}.json'.format(prop_lists_json_path, city_name, year)
        with open(file_path, 'wb') as f:
            f.write(data)

def convert_prop_listing_json_data_to_csv():
    setUp()
    unwanted_prop_keys = ['occ', 'revenue', 'regions', 'id', 'platforms', 'homeaway_property_id', 'm_homeaway_property_id']
    for city_id, city_name in city_id_dict.items():
        file_path = '{}{}_property_list_{}.json'.format(prop_lists_json_path, city_name, year)
        with open(file_path) as f:
            data = json.load(f)
        property_data = data['properties']
        # Clean data and add additional calculated fields
        for prop in property_data:
            # Additional columns
            prop['airbnb_url'] = 'https://www.airbnb.com/rooms/' + str(prop['airbnb_property_id'])
            # Check & record if Airbnb listing is active
            prop['active_listing'] = 0
            if (requests.head(prop['airbnb_url']).status_code == 200):
                prop['active_listing'] = 1
            prop['market'] = markets[city_name]
            # Filter out unwanted property keys
            for unwanted_prop_key in unwanted_prop_keys:
                prop.pop(unwanted_prop_key)

        # Write data to csv
        csv_file_path = '{}{}_property_list.csv'.format(prop_lists_csv_path, city_name)
        data_file = open(csv_file_path, 'w', encoding='utf-8', newline='')
        csv_writer = csv.writer(data_file)
        count = 0

        for prop in property_data: 
            if count == 0: 
                # Write headers
                header = prop.keys() 
                csv_writer.writerow(header) 
                count += 1
            # Write data 
            csv_writer.writerow(prop.values()) 

        data_file.close()

# fetch_prop_listing_json_files()
convert_prop_listing_json_data_to_csv()
# print('Completed')
# print(datetime.datetime.now() - begin_time)

From cffi callback <function _verify_callback at 0x0000008864D41828>:
Traceback (most recent call last):
  File "C:\Users\david\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='www.airbnb.com', port=443): Max retries exceeded with url: /rooms/32637974 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [ ]:
# Objective: Combine all dfw market property listings into a single file

import pandas as pd
import plotly.express as px

markets = {
    'dallas' : 'Dallas',
    'fort-worth' : 'Fort-Worth',
    'plano' : 'Plano',
    'irving' : 'Irving',
    'arlington' : 'Arlington',
    'frisco' : 'Frisco',
    'grand-prairie' : 'Grand Prairie',
    'mckinney' : 'McKinney',
    'denton' : 'Denton',    
    'richardson' : 'Richardson'
}
prop_lists_csv_path = 'prop_lists/csv/'

df = pd.DataFrame()
for market_id, market_name in markets.items():
    file_path = '{}{}_property_list_2019.csv'.format(prop_lists_csv_path, market_id)
    df = df.append(pd.read_csv(file_path))

# Merge all market prop listings to a single file
df.to_csv('prop_lists/csv/all_market_property_list_2019.csv', index=False)

In [4]:
# Objective: Combine Airbnb Texas rental data with Airdna DFW Property listings

import pandas as pd
import requests

airbnb_texas_rental_data = 'data/Airbnb_texas_rental_data.csv'
df1 = pd.read_csv(airbnb_texas_rental_data)
df1 = df1[
    df1.city.str.contains('Dallas', case=False)
    | df1.city.str.contains('Fort Worth', case=False)
    | df1.city.str.contains('Plano', case=False)
    | df1.city.str.contains('Irving', case=False)
    | df1.city.str.contains('Arlington', case=False)
    | df1.city.str.contains('Frisco', case=False)
    | df1.city.str.contains('Grand Prairie', case=False)
    | df1.city.str.contains('McKinney', case=False)
    | df1.city.str.contains('Denton', case=False)
    | df1.city.str.contains('Richardson', case=False)
]
df1 = df1[df1.city != 'Lake Dallas']
df1['average_daily_rate'] = df1['average_daily_rate'].str.replace('$', '')
df1['city'] = df1['city'].str.replace('Fort Worth', 'Fort-Worth', case=False)
df1['bedrooms'] = df1['bedrooms'].str.replace('Studio', '0')
df1.head()

,average_daily_rate,bedrooms,city,listing_date,description,latitude,longitude,title,url
4,75,2,Fort-Worth,Feb-17,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...
7,25,1,Fort-Worth,Jan-16,This is a beautiful bedroom with a queen size ...,32.689611,-97.298161,Friendly Private Room in َQuiet Neighborhood,https://www.airbnb.com/rooms/18977363?location...
10,65,1,Irving,Jul-15,My cool and comfortable bedroom apartment feel...,32.862116,-97.002643,Cozy 1 bedroom/bathroom with pool,https://www.airbnb.com/rooms/7276294?location=...
19,250,2,Frisco,Dec-14,This unique lakefront area is perfect for crea...,33.155776,-96.930438,Lake Lewisville Lakefront Convenient to Frisco,https://www.airbnb.com/rooms/4785810?location=...
24,100,0,Denton,Nov-15,A converted carriage house walking distance to...,33.216140,-97.137757,Historic Converted Carriage House,https://www.airbnb.com/rooms/16650025?location...


In [5]:
all_property_list = 'data/prop_lists/csv/all_market_property_list_2019.csv'
df2 = pd.read_csv(all_property_list)
df2 = df2.rename(columns={'market': 'city', 'adr': 'average_daily_rate', 'airbnb_url' : 'url'})
df2['average_daily_rate'] = round(df2['average_daily_rate'])
del df2['homeaway_property_id']
del df2['m_homeaway_property_id']
df2.head()

,airbnb_property_id,rating,property_type,bathrooms,title,bedrooms,accommodates,average_daily_rate,img_cover,reviews,longitude,latitude,days_available,room_type,url,active_listing,city
0,35006003,9.3,House,2.0,"Mixed/Co-Ed Dorm, Bed #D7 (Bottom Bunk) for No...",1,1,23.0,https://a0.muscache.com/im/pictures/db46db81-c...,9,-96.82623,32.75400,205,Shared room,https://www.airbnb.com/rooms/35006003,1,Dallas
1,34289001,9.6,Apartment,1.0,Entire Apt- Enhanced cleaning/sanitizing routines,1,2,75.0,https://a0.muscache.com/im/pictures/83c78201-5...,57,-96.81452,32.80762,315,Entire home/apt,https://www.airbnb.com/rooms/34289001,1,Dallas
2,27929626,10.0,Apartment,1.0,⭐️⭐️⭐️⭐️Huge 1 BD + Den AIRBNB DALLAS MONTHLY,1,4,112.0,https://a0.muscache.com/im/pictures/adf7dab5-9...,3,-96.83065,32.81013,122,Entire home/apt,https://www.airbnb.com/rooms/27929626,1,Dallas
3,38355276,9.0,Apartment,1.0,COMFY APARTMENT IN DOWNTOWN DALLAS NEAR AIRPORT,1,3,102.0,https://a0.muscache.com/im/pictures/c95cbae7-7...,2,-96.82262,32.81917,161,Entire home/apt,https://www.airbnb.com/rooms/38355276,1,Dallas
4,36914040,8.0,House,2.0,"R432 Low Cost Private Room,Single guest only",1,1,33.0,https://a0.muscache.com/im/pictures/aacb109b-2...,1,-96.74533,32.67270,182,Private room,https://www.airbnb.com/rooms/36914040,1,Dallas


In [6]:
df3 = pd.concat([df2, df1], ignore_index=True)
# df3_csv = df3.to_csv(index=False)
df3.to_csv(r'data/airbnb_dfw_property_listings.csv', index = False)

# import csv

# csv_file_path = 'airbnb_texas_rental_data_combined.csv'
# data_file = open(csv_file_path, 'w', encoding='utf-8', newline='')
# csv_writer = csv.writer(df3_csv)